In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import time

from scripts.ImageDataset import ImageDataset
from models.FoodCNN import FoodCNN
from utils.LogClass import LogClass

def get_accuracy(prediction, gt_labels, percentage=True):
    correct = 0
    for i in range(len(prediction)):
        if percentage:
            label_prediction = np.argmax(prediction[i])
        else:
            label_prediction = prediction[i]
        if label_prediction == gt_labels[i]:
            correct += 1
    return correct / len(prediction)

# Configurazione del dataset e dei DataLoader (come nel tuo codice)
train_gt_path = './ground_truth/train_small.csv'
test_gt_path = './ground_truth/new_val_info.csv'

train_image_path = './train_set/'
test_image_path = './val_set/'

train_df = pd.read_csv(train_gt_path, header=None, names=['image_id', 'label'])
test_df = pd.read_csv(test_gt_path, header=None, names=['image_id', 'label'])

train_dataset = ImageDataset(train_df, train_image_path, train=True)
test_dataset = ImageDataset(test_df, test_image_path, train=False)
lc = LogClass('technical_report.txt')
model_name = 'resnet18'
gt_labels = test_dataset.get_all_labels()

In [3]:
model = FoodCNN(model_name=model_name, num_classes=251, device='cuda')
start_train = time.time()
train_loss, val_loss, train_accuracies, val_accuracies = model.train_model(train_dataset, validation=0.1, num_epochs=20, lr=0.001)
end_train = time.time()

print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
lc.write(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
print(f"Training time: {end_train - start_train:.2f} s")
lc.write(f"Training time: {end_train - start_train:.2f} s")

Epoch 1/20:   8%|█▊                    | 12/142 [00:05<00:59,  2.19it/s, accuracy=0.00781, loss=6.1]


KeyboardInterrupt: 

In [ ]:

# Plot delle loss e delle accuratezze
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.savefig(f'graph/{model_name}_loss_plot.png')

plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.savefig(f'graph/{model_name}_accuracy_plot.png')

start_predict = time.time()
prediction = model.predict(test_dataset)
end_predict = time.time()

print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
lc.write(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
print(f"Prediction time: {end_predict - start_predict:.2f} s")
lc.write(f"Prediction time: {end_predict - start_predict:.2f} s")

accuracy = get_accuracy(prediction, gt_labels)
print(f'Accuracy: {accuracy}')
lc.write(f'Accuracy: {accuracy}')
lc.write('----------------------------------------------------------------------------------------------------------\n')
lc.close()